In [59]:
import os
import sys
import datetime

#图像读取库
from PIL import Image
#矩阵运算库
import numpy as np
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing import image

print(tf.__version__)
print(keras.__version__, keras.__path__)

2.3.0
2.4.0 ['/home/wilson/.local/lib/python3.8/site-packages/tensorflow/keras']


In [60]:
# 获取训练的图片列表和分类标签信息
def load_images(data_dir):
    fpaths = []
    labels = []
    
    for label in os.listdir(data_dir):
        labelPath = os.path.join(data_dir, label)
        for fname in os.listdir(labelPath):
            # 文件名
            fpath = os.path.join(labelPath, fname)
            fpaths.append(fpath)
            
            # 标签
            labels.append(label_to_index[label])
    
    # 将图片名和标签 打包成一组，然后进行乱序
    temp = np.array([fpaths,labels])
    temp = temp.transpose()
    np.random.shuffle(temp)

    fpaths = list(temp[:,0])
    labels = list(temp[:,1])
    labels = [int(i) for i in labels]
    
    return fpaths, labels

def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image,dtype=tf.float32)
    image /= 255.0
    #print(333, image.shape, image)
    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

In [61]:
# 模型保存位置
model_save_path = 'emotion.model'

# 加载模型
model = None
if not os.path.exists(model_save_path):
    print("failed to find saved model in:", model_save_path)
    sys.exit(1)
else:
    model = keras.models.load_model(model_save_path)
    
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_192 (Functi (None, 6, 6, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 2,230,277
Non-trainable params: 34,112
_________________________________________________________________


In [62]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# 分类
label_names = ["anger", "disgust", "fear", "happy", "sad"]
label_to_index = dict((name, index) for index, name in enumerate(label_names))

print(label_names)
print(label_to_index)

# 准备测试数据
data_dir = "/home/wilson/workstation/TFResFaceEmotion/emotions"

all_image_paths, all_image_labels = load_images(data_dir)
#print("all_image_lavels", all_image_labels)
#print("all_image_paths", all_image_paths)
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)
final_ds = image_ds.shuffle(buffer_size=10).batch(10)

print("ccccc", final_ds)

predict = model.predict(final_ds)
print("ddddd", predict)

['anger', 'disgust', 'fear', 'happy', 'sad']
{'anger': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'sad': 4}
ccccc <BatchDataset shapes: (None, None, None, 3), types: tf.float32>
ddddd [[1.04499221e-01 8.47925246e-02 1.27792329e-01 2.52752870e-01
  4.30163056e-01]
 [9.17759955e-01 8.56552260e-11 9.63333528e-04 8.03727880e-02
  9.03916603e-04]
 [1.04209213e-02 1.16061524e-03 7.41047179e-03 4.00076173e-02
  9.41000342e-01]
 [5.01882613e-01 7.39134327e-02 1.98168054e-01 2.71187257e-02
  1.98917150e-01]
 [5.73063374e-01 8.98130331e-03 1.63334742e-01 1.18355274e-01
  1.36265337e-01]
 [5.75805716e-02 2.08110170e-04 8.04055929e-01 9.58382487e-02
  4.23170589e-02]
 [2.02629473e-02 2.59468006e-03 1.83438081e-02 2.71831483e-01
  6.86967015e-01]
 [6.30281866e-02 3.30061726e-02 3.75385433e-02 3.38454209e-02
  8.32581639e-01]
 [7.60683894e-01 2.62498594e-04 5.41784540e-02 9.47547182e-02
  9.01204720e-02]
 [5.00513939e-04 8.95818870e-04 1.24834070e-03 1.44744851e-02
  9.82880831e-01]
 [8.58945370e-01 3.